**How can do NLP**

* Firstly we throw punctuation("",.:;)
* Then we split all word
* Then we return all word lower case
* We find word root with lemmatizer
* We make bag of word(It is including the most using word and sentence's info)
* We make prediction model 

In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir())

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', 'embeddings.ipynb', 'glove.6B.100d.txt', 'LDA_viz_tsne.ipynb', 'NB-LR-DL.ipynb', 'spam.csv']


In [42]:
#Firstly we read our data
data = pd.read_csv("spam.csv", encoding = "ISO-8859-1")
data = data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)
data = data.rename({"v1":"Category","v2":"Message"},axis=1)
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [43]:
#We must change category 1 or 0
data["Category"] = [1 if each == "spam" else 0 for each in data["Category"]]

In [44]:
data.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


**Example**

It is example for you learn.I will do one row but ı will have all row in below

In [51]:
#We choose 1 row.And we throw punctuation
import re
nlp_data = str(data.iloc[2,:])
nlp_data = re.sub("[^a-zA-Z]"," ",nlp_data)

In [52]:
#After return lower case
nlp_data = nlp_data.lower()

In [56]:
#we have two choice we can use split methot or tokenize
import nltk as nlp
nlp_data = nlp.word_tokenize(nlp_data)
#nlp_data = nlp_data.split() or we can do so
nlp_data

['category',
 'message',
 'free',
 'entry',
 'in',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'fina',
 'name',
 'dtype',
 'object']

In [57]:
#we have to find word root
lemma = nlp.WordNetLemmatizer()
nlp_data = [lemma.lemmatize(word) for word in nlp_data]
nlp_data

['category',
 'message',
 'free',
 'entry',
 'in',
 'a',
 'wkly',
 'comp',
 'to',
 'win',
 'fa',
 'cup',
 'fina',
 'name',
 'dtype',
 'object']

In [58]:
#We join our data
nlp_data = " ".join(nlp_data)
nlp_data

'category message free entry in a wkly comp to win fa cup fina name dtype object'

And we have list is including all word now we have to prepare predict model but we will do the rest in below

**NLP**

We do this process to all data with for loop

In [59]:
import nltk as nlp
import re
description_list = []
for description in data["Message"]:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   # buyuk harftan kucuk harfe cevirme
    description = nlp.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words("english"))]
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description) #we hide all word one section

In [60]:
#We make bag of word it is including number of all word's info
from sklearn.feature_extraction.text import CountVectorizer 
max_features = 3000 #We use the most common word
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("the most using {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

the most using 3000 words: ['aah', 'aathi', 'abi', 'ability', 'abiola', 'abj', 'able', 'absolutly', 'abt', 'abta', 'aburo', 'ac', 'academic', 'acc', 'accept', 'access', 'accident', 'accidentally', 'accordingly', 'account', 'ache', 'acl', 'aco', 'acted', 'action', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'adam', 'add', 'addamsfa', 'added', 'addicted', 'addie', 'address', 'admin', 'administrator', 'admirer', 'admit', 'adore', 'adoring', 'adult', 'advance', 'adventure', 'advice', 'advise', 'ae', 'aeronautics', 'aeroplane', 'affair', 'affection', 'afraid', 'aft', 'afternoon', 'aftr', 'ag', 'agalla', 'age', 'agent', 'ago', 'agree', 'ah', 'aha', 'ahead', 'ahmad', 'aid', 'aight', 'ain', 'aint', 'air', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'aj', 'aka', 'al', 'alaipayuthe', 'album', 'alcohol', 'alert', 'alex', 'alfie', 'algarve', 'ali', 'alive', 'allah', 'allow', 'allowed', 'alright', 'alrite', 'alwys', 'amazing', 'american', 'amp', 'amt', 'amused'

In [61]:
#We separate our data is train and test
y = data.iloc[:,0].values   # male or female classes
x = sparce_matrix
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [62]:
#We make model for predict
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
print("the accuracy of our model: {}".format(nb.score(x_test,y_test)))

the accuracy of our model: 0.8763440860215054


In [63]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("our accuracy is: {}".format(lr.score(x_test,y_test)))

C:\Users\EtienneHerlaut\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


our accuracy is: 0.9838709677419355


In [64]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
#print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn.score(x_test,y_test))

With KNN (K=3) accuracy is:  0.9587813620071685


In [65]:
x_test = x_test.reshape(558,3000,1)

In [66]:
x_train = x_train.reshape(5014,3000,1)

In [67]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

# Fitting the RNN to the Training set
regressor.fit(x_test, y_test, epochs = 3, batch_size = 32)


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
558/558 [==============================] - 238s 427ms/step - loss: 0.1083 - acc: 0.8799
Epoch 2/3
558/558 [==============================] - 254s 455ms/step - loss: 0.1069 - acc: 0.8799
Epoch 3/3
558/558 [==============================] - 247s 443ms/step - loss: 0.1062 - acc: 0.8799
